In [28]:
import numpy as np
from typing import Tuple
from tensorflow.keras.datasets import boston_housing
from tensorflow.keras.initializers import Constant
from tensorflow.keras.initializers import RandomUniform
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential

In [29]:
def get_dataset() -> Tuple[Tuple[np.ndarray, np.ndarray], Tuple[np.ndarray, np.ndarray]]:
    (x_train, y_train), (x_test, y_test) = boston_housing.load_data()
    x_train = x_train.astype(np.float32)
    y_train = y_train.reshape((-1, 1)).astype(np.float32)
    x_test = x_test.astype(np.float32)
    y_test = y_test.reshape((-1, 1)).astype(np.float32)

    return (x_train, y_train), (x_test, y_test)

In [30]:
def build_model(num_features: int, num_targets: int) -> Sequential:
    init_w = RandomUniform(minval=-1.0, maxval=1.0)
    init_b = Constant(value=0.0)

    model = Sequential()
    model.add(Dense(units=16, kernel_initializer=init_w, bias_initializer=init_b, input_shape=(num_features,))) 
        # Kernel waere die W-, bias die b-Matrix (Weights und Bias)
        # Müssen nicht angegeben werden, weil per Default praktische Zufallswerte gewählt werden
    model.add(Activation('relu'))
    model.add(Dense(units=num_targets))
    model.summary()
    return model

In [31]:
def main() -> None:
    (x_train, y_train), (x_test, y_test) = get_dataset()

    print(f'x train shape: {x_train.shape}')
    print(f'y train shape: {y_train.shape}')
    print(f'x test shape: {x_test.shape}')
    print(f'y test shape: {y_test.shape}')
    
    num_features = 13
    num_targets = 1

    model = build_model(num_features, num_targets)

In [32]:
if __name__=='__main__':
    main()

x train shape: (404, 13)
y train shape: (404, 1)
x test shape: (102, 13)
y test shape: (102, 1)
Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_2 (Dense)             (None, 16)                224       
                                                                 
 activation_1 (Activation)   (None, 16)                0         
                                                                 
 dense_3 (Dense)             (None, 1)                 17        
                                                                 
Total params: 241
Trainable params: 241
Non-trainable params: 0
_________________________________________________________________
